In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Dense, Input, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from keras.utils import plot_model
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import mat73

Using TensorFlow backend.


In [9]:
feat = pd.read_csv('5features.csv')

In [6]:
#target = data.pop('Systolic')
#target2 = data.pop('Diastolic')
#dataset = tf.data.Dataset.from_tensor_slices((data.values, target.values))
#for feature, target in dataset.take(5):
    #print('Features: {}, Target: {}'.format(feature,target))

In [10]:
x = feat.values[:,0:5]
y = feat.values[:,5:]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size = 0.5)

In [11]:
y_train.shape

(710, 2)

In [12]:
inputs = Input(shape=(5), name='Input')
hidden = Dense(20, name = 'Hidden', activation='linear')(inputs)
hidden1 = Dense(45, name = 'Hidden1', activation='relu')(hidden)
hidden2 = Dense(60, name ='Hidden2', activation='linear')(hidden1)
hidden3 = Dense(20, name = 'Hidden3', activation='relu')(hidden2)
hidden4 = Dense(5, name = 'Hidden4', activation='relu')(hidden3)
output = Dense(2, name='Output')(hidden4)
model = Model(inputs = inputs, outputs = output)
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 5)]               0         
_________________________________________________________________
Hidden (Dense)               (None, 20)                120       
_________________________________________________________________
Hidden1 (Dense)              (None, 45)                945       
_________________________________________________________________
Hidden2 (Dense)              (None, 60)                2760      
_________________________________________________________________
Hidden3 (Dense)              (None, 20)                1220      
_________________________________________________________________
Hidden4 (Dense)              (None, 5)                 105       
_________________________________________________________________
Output (Dense)               (None, 2)                

In [ ]:
#model1 = Sequential()
#model1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=((x_train.shape[0],x_train.shape[1]))))
#model1.add(Conv1D(filters=64, kernel_size=3, activation='relu'))

In [ ]:
model.compile(optimizer="adam", loss="mse")

In [ ]:
model.fit(x_train, y_train, batch_size = 3, epochs=10)

In [56]:
model.evaluate(x_test, y_test)

5/5 [==============================] - 0s 1ms/step - loss: 107.3783


107.37828063964844

In [43]:
predict = model.predict(x_train)
print(predict)

[[131.21742   71.50149 ]
 [ 95.87009   52.036213]
 [134.27281   73.14973 ]
 ...
 [133.4282    72.70058 ]
 [132.3035    72.06907 ]
 [130.35028   71.03007 ]]


In [17]:
y_train

array([[135.34701294,  75.21992058],
       [130.56029072,  73.75459745],
       [135.98198629,  74.78032364],
       ...,
       [130.95104355,  72.0450538 ],
       [130.56029072,  71.60545686],
       [124.06402485,  67.79561673]])

In [1]:
# Another model, convolutional that will take the PPG signal, the first derivative, and the second derivative
# First up is the PPG signal neural network 
# Use Conv1D then maxpooling layers 3 in a row at least

In [4]:
# Extract PPG data into CSV files
orig = mat73.loadmat(r'data/Part_1.mat')

In [5]:
ppg = orig['Part_1'][0][0]
abp = orig['Part_1'][0][1]

In [6]:
len(ppg)
len(abp)

61000

In [7]:
# Export the current feature list into CSV file # DONT RUN AGAIN ALREADY HAVE FILE
df = pd.DataFrame({'y': ppg, 'ABP': abp})
df.to_csv("smallPPG.csv", index=True, index_label='x')

In [8]:
# Read new CSV file with PPG and ABP
data = pd.read_csv('smallPPG.csv')
len(data)


61000

In [9]:
data

,x,y,ABP
0,0,1.759531,67.062955
1,1,1.718475,69.358628
2,2,1.684262,75.366453
3,3,1.657869,85.037586
4,4,1.637341,96.222885
...,...,...,...
60995,60995,1.771261,74.438415
60996,60996,1.700880,74.536103
60997,60997,1.644184,76.441023
60998,60998,1.600196,81.813875


In [10]:
Scaler = MinMaxScaler()
Scaler.fit_transform(data)

array([[0.00000000e+00, 4.39667807e-01, 1.53709462e-01],
       [1.63937114e-05, 4.29408891e-01, 1.74589072e-01],
       [3.27874227e-05, 4.20859795e-01, 2.29231453e-01],
       ...,
       [9.99967213e-01, 4.10845139e-01, 2.39004887e-01],
       [9.99983606e-01, 3.99853444e-01, 2.87872057e-01],
       [1.00000000e+00, 3.91304348e-01, 3.74944469e-01]])

In [37]:
x1 = data.values[:,0:2]
y1 = data.values[:,2:]
x1 = np.expand_dims(x1, axis=2)

x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size = 0.3)
x_valid1, x_test1, y_valid1, y_test1 = train_test_split(x_test1, y_test1, test_size = 0.5)

In [43]:
y_train1.shape

(42700, 1)

In [39]:
n_timesteps, n_features, n_outputs = x_train1.shape[1], x_train1.shape[2], y_train1.shape[1]

In [40]:
inputs = Input(shape=(n_timesteps, n_features), name='Input')
conv1 = Conv1D(filters = 64, kernel_size = 1, activation = 'relu')(inputs)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPooling1D(pool_size = 2)(drop1)
flat1 = Flatten()(pool1)
dense1 = Dense(100, activation='relu')(flat1)
output = Dense(n_outputs, activation='relu')(dense1)
model = Model(inputs = inputs, outputs = output)
print(model.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 2, 1)]            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 2, 64)             128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 64)             0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               6500      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [41]:
model.compile(optimizer="adam", loss="mse")

In [42]:
model.fit(x_train1, y_train1, batch_size = 1, epochs=10)

Epoch 1/10
42700/42700 [==============================] - 67s 2ms/step - loss: 9492.6865
Epoch 2/10
42700/42700 [==============================] - 68s 2ms/step - loss: 9379.0078
Epoch 3/10
42700/42700 [==============================] - 69s 2ms/step - loss: 9379.0645
Epoch 4/10
42700/42700 [==============================] - 69s 2ms/step - loss: 9379.0059
Epoch 5/10
42700/42700 [==============================] - 69s 2ms/step - loss: 9379.0566
Epoch 6/10
42700/42700 [==============================] - 69s 2ms/step - loss: 9379.0342
Epoch 7/10
42700/42700 [==============================] - 69s 2ms/step - loss: 9379.0342
Epoch 8/10
42700/42700 [==============================] - 69s 2ms/step - loss: 9378.9746
Epoch 9/10
42700/42700 [==============================] - 69s 2ms/step - loss: 9379.0010
Epoch 10/10
42700/42700 [==============================] - 69s 2ms/step - loss: 9379.0010


In [33]:
# Differentiate the PPG data (first derivative)
dy = np.diff(ppg)
len(dy)

60999

In [34]:
# Differentiatea the PPG data again (second derivative)
dyy = np.diff(dy)

In [36]:
x_train1.shape

(42700, 2)

In [3]:
ppg_window = np.load('ppg_windows.npy')

In [4]:
sbp = np.load('sbp.npy')
dbp = np.load('dbp.npy')

In [5]:
sbp

array([122.01257247, 120.96242423, 120.88915807, 125.18743925,
       123.18483097, 125.23628335, 126.2864316 , 124.60131   ,
       126.40854186, 126.3352757 , 122.67196788, 122.06141658,
       122.06141658, 121.67066374, 125.43165977, 124.35708948,
       127.89828704, 129.2414999 , 128.38672808, 131.31737434,
       125.33397156, 125.92010081, 130.07184968, 125.43165977,
       129.55898658, 129.09496759, 125.43165977, 126.21316544,
       128.14250756, 128.28903987, 130.85335535, 126.3841198 ,
       126.67718443, 126.50623007, 125.48050388, 129.82762916,
       126.67718443, 128.80190297, 129.29034401, 127.97155319,
       130.3160702 , 129.46129837, 127.75175472, 128.21577372,
       127.28773573, 124.40593358, 127.99597525, 125.13859515,
       129.09496759, 129.8032071 , 128.28903987, 130.65797893,
       128.5821045 , 125.55377003, 127.40984599, 126.79929469,
       124.57688795, 127.09235932, 123.08714277, 124.45477769,
       126.01778902, 123.89307049, 125.67588029, 128.82

In [14]:
x2 = ppg_window
y2 = sbp
x2 = np.expand_dims(x2, axis=2)
y2 = np.expand_dims(y2, axis=1)

x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size = 0.3)
x_valid2, x_test2, y_valid2, y_test2 = train_test_split(x_test2, y_test2, test_size = 0.5)

In [15]:
np.shape(y_train2)

(236, 1)

In [16]:
n_timesteps, n_features, n_outputs = x_train2.shape[1], x_train2.shape[2], y_train2.shape[1]

In [18]:
inputs = Input(shape=(n_timesteps, n_features), name='Input')
conv1 = Conv1D(filters = 64, kernel_size = 1, activation = 'relu')(inputs)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPooling1D(pool_size = 2)(drop1)
flat1 = Flatten()(pool1)
dense1 = Dense(100, activation='relu')(flat1)
output = Dense(n_outputs, activation='relu')(dense1)
window = Model(inputs = inputs, outputs = output)
print(window.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 180, 1)]          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 180, 64)           128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 180, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 90, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5760)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               576100    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [27]:
window.compile(optimizer="adam", loss="mse")
window.fit(x_train2, y_train2, batch_size = 1, epochs=10)

Epoch 1/10
236/236 [==============================] - 1s 3ms/step - loss: 98.9247
Epoch 2/10
236/236 [==============================] - 1s 3ms/step - loss: 89.8262
Epoch 3/10
236/236 [==============================] - 1s 3ms/step - loss: 77.3451
Epoch 4/10
236/236 [==============================] - 1s 3ms/step - loss: 86.0157
Epoch 5/10
236/236 [==============================] - 1s 3ms/step - loss: 75.2363
Epoch 6/10
236/236 [==============================] - 1s 3ms/step - loss: 76.5552
Epoch 7/10
236/236 [==============================] - 1s 3ms/step - loss: 92.2395
Epoch 8/10
236/236 [==============================] - 1s 3ms/step - loss: 74.1273
Epoch 9/10
236/236 [==============================] - 1s 3ms/step - loss: 91.1290
Epoch 10/10
236/236 [==============================] - 1s 3ms/step - loss: 78.9177


In [21]:
x3 = ppg_window
y3 = dbp
x3 = np.expand_dims(x3, axis=2)
y3 = np.expand_dims(y3, axis=1)

x_train3, x_test3, y_train3, y_test3 = train_test_split(x3, y3, test_size = 0.3)
x_valid3, x_test3, y_valid3, y_test3 = train_test_split(x_test3, y_test3, test_size = 0.5)

In [22]:
n_timesteps, n_features, n_outputs = x_train2.shape[1], x_train2.shape[2], y_train2.shape[1]

In [23]:
inputs = Input(shape=(n_timesteps, n_features), name='Input')
conv1 = Conv1D(filters = 64, kernel_size = 1, activation = 'relu')(inputs)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPooling1D(pool_size = 2)(drop1)
flat1 = Flatten()(pool1)
dense1 = Dense(100, activation='relu')(flat1)
output = Dense(n_outputs, activation='relu')(dense1)
window2 = Model(inputs = inputs, outputs = output)
print(window.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 180, 1)]          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 180, 64)           128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 180, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 90, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5760)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               576100    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [26]:
window2.compile(optimizer="adam", loss="mse")
window2.fit(x_train3, y_train3, batch_size = 1, epochs=100)

Epoch 1/100
236/236 [==============================] - 1s 3ms/step - loss: 35.7722
Epoch 2/100
236/236 [==============================] - 1s 3ms/step - loss: 33.9273
Epoch 3/100
236/236 [==============================] - 1s 3ms/step - loss: 32.8852
Epoch 4/100
236/236 [==============================] - 1s 3ms/step - loss: 29.3792
Epoch 5/100
236/236 [==============================] - 1s 3ms/step - loss: 29.0187
Epoch 6/100
236/236 [==============================] - 1s 3ms/step - loss: 26.8564
Epoch 7/100
236/236 [==============================] - 1s 3ms/step - loss: 30.1036
Epoch 8/100
236/236 [==============================] - 1s 3ms/step - loss: 30.1050
Epoch 9/100
236/236 [==============================] - 1s 3ms/step - loss: 24.1083
Epoch 10/100
236/236 [==============================] - 1s 3ms/step - loss: 24.2886
Epoch 11/100
236/236 [==============================] - 1s 3ms/step - loss: 27.2098
Epoch 12/100
236/236 [==============================] - 1s 3ms/step - loss: 19.7251
E

236/236 [==============================] - 1s 3ms/step - loss: 16.4450
Epoch 99/100
236/236 [==============================] - 1s 3ms/step - loss: 17.1688
Epoch 100/100
236/236 [==============================] - 1s 3ms/step - loss: 18.6061
